 ## ASGI

ASGI (async server gateway interface) is a successor (kinda) to WSGI, it's async and not tied to the request-response paradigm.

You need to provide a function taking 3 arguments: `scope`, `receive`, and `send`.

* `scope` is information about current connection.
* `receive` is a coroutine that can, ahem, receive event from the client
* `send` is a coroutine that can, ahem, send event to the client.

Event can be e.g., http request, http response, chunk in http response, or websocket message.

ASGI server will call your function once for every http request or websocket connection.

In [5]:
import json


async def app(scope, receive, send):
    # type is a required key in scope
    if scope["type"] == "http":
        # scope contains information about http request (path, headers, etc)
        print(f'got {scope=}')
        await send({'type': 'http.response.start', 'status': 200, 'headers': [[b'content-type', b'application/json']]})
        await send(
            {'type': 'http.response.body', 'body': json.dumps({"healthy": "ok"}).encode('ascii'), 'more_body': False})
    elif scope["type"] == "websocket":
        # websocket protocol in asgi is this:
        # we get event type=websocket.connect
        event = await receive()
        print(f'got {event=}')
        # we respond with websocket.accept
        await send({'type': 'websocket.accept'})
        # now we can receive messages with `receive` (type=websocket.receive)
        event = await receive()
        print(f'got {event=}')
        # and send messages with `send` type='websocket.send'
        await send({'type': 'websocket.send', 'text': f'hello from websocket {event}'})
    else:
        print(f"got {scope=}")


We can run our app with ASGI-compatible server. Cool kids use `uvicorn` as an ASGI-compatible server.


In [ ]:
import functools
import threading
import uvicorn

# running uvicorn in a separate thread, to work around asyncio.run() inside of asyncio.run() 
# lifespan='on' enables asgi lifespan spec: https://asgi.readthedocs.io/en/latest/specs/lifespan.html
# this is probably what fastapi uses for its lifespan events (startup/shutdown/etc)
run_uvicorn = functools.partial(uvicorn.run, host='0.0.0.0', port=8000, lifespan='on')
t = threading.Thread(target=run_uvicorn, args=(app,))
t.start()
t.join()

INFO:     Started server process [75574]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


got scope={'type': 'lifespan', 'asgi': {'version': '3.0', 'spec_version': '2.0'}, 'state': {}}
got scope={'type': 'http', 'asgi': {'version': '3.0', 'spec_version': '2.3'}, 'http_version': '1.1', 'server': ('127.0.0.1', 8000), 'client': ('127.0.0.1', 50101), 'scheme': 'http', 'root_path': '', 'headers': [(b'host', b'localhost:8000'), (b'user-agent', b'curl/8.6.0'), (b'accept', b'*/*')], 'state': {}, 'method': 'GET', 'path': '/', 'raw_path': b'/', 'query_string': b''}
INFO:     127.0.0.1:50101 - "GET / HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 50184) - "WebSocket /websockets_stream" [accepted]
INFO:     connection open
INFO:     connection closed


got event={'type': 'websocket.connect'}
got event={'type': 'websocket.receive', 'text': '{"some": "message"}'}


Let's make http request to our server

```shell
curl http://localhost:8000
```

For websockets example use websocket request from [intellij_http_client.http](./intellij_http_client.http)

ASGI is extensible, you can extend it to support GRPC etc. You just need a support from ASGI-server for that.